# Psuedo-Surge Simulation of Synthetic Little Kluane  


Before we figure out the best way to prescribe sliding in `Elmer` let's first take some time to understand how sliding in prescribed in `Elmer`. 
In `Elmer/Ice` the default sliding law is of the form: 
$$
\tau = \beta^2 u
$$
where $\tau$ is traction (i.e. basal shear stress), $u$ is velocity in the tangetial plane, and $\beta^{-2}$ is the slip coefficent. Let's use some example values from Cuffey and Patterson to look into units, and how we will need to scale to `Elmer/Ice` base units of MPa, a, m. 
Note, Cuffey and Patterson use a simpler measure of lubrication with an apparent drag factor, $\psi$, a positive number
defined by
$$
\tau_b = \psi u_b
$$
for a basal shear stress $\tau_b$, and rate of slip $u_b$.
Values listed below come from Table 7.2 in Cuffey and Patterson

|    Glacier    | $\tau_b$ <br>(kPa) | $u_b$ <br>(m y$^{-1}$) | $\psi$<br>(kPa (m y$^{-1}$)$^{-1}$) |
|:-------------:|:------------------:|:----------------------:|:-----------------------------------:|
|   Trapridge   |         80         |           30           |        $\approx$ 3                  |
| Storglaciären |         40         |           30           |        $\approx$ 1                  |

So, let's begin by scaling these example parameter values to `Elmer/Ice` base units (MPa, a, m): 

|    Glacier    | $\tau_b$ <br>(MPa) | $u_b$ <br>(m y$^{-1}$) | $\psi$<br>(MPa (m y$^{-1}$)$^{-1}$) |
|:-------------:|:------------------:|:----------------------:|:-----------------------------------:|
|   Trapridge   |         8e-2       |           30           |       $\approx$ 3e-3                |
| Storglaciären |         4e-2       |           30           |       $\approx$ 1e-3                |


Great! Now we can see from these two simple formulations: 
$$
\beta = \sqrt{\psi}
$$

Let's use our inital conditions (i.e. $z_s$ and $z_b$) to figure out the corresponding value of $\beta$ for some set value of $u_b$. To do this we will need to make an approximation of $\tau_b$ using the approximation: 

$$
\tau_b = \rho g H \sin \alpha
$$

where $\rho$ is the ice density, $g$ is the acceleration due to gravity, and $\alpha$ is the suface slope. 

First, lets make sure our units are going to be correct during these back of the envelope calculations: 

In [1]:
import pint 
import numpy as np 
import matplotlib.pyplot as plt 

plt.rcParams['text.usetex'] = True

ureg = pint.UnitRegistry()

#############################################
# units
#############################################
a   = ureg.year             # [a]
s   = ureg.sec              # [s]
m   = ureg.meter            # [m]
kg  = ureg.kilogram         # [kg]
kPa = ureg('kPa')           # [kPa]
MPa = ureg('MPa')           # [MPa]

#############################################
# parameters (S.I. units)
#############################################
spy   = 365.25*24*60*60 * (s/a)                    # [s a^-1]
ρ     = 910             * (kg*m**-3)               # [Kg m^-3]
g     = 9.81            * (m*s**-2)                # [m s^-2]

#############################################
# parameters (Elmer/Ice)
#############################################
ρ     = ρ * (MPa/MPa.to_base_units()) * spy**-2    # [MPa m^-2 a^2] <--[Kg m^-3]
g     = g * spy**2                                 # [m a^-2] <--------[m s^-2]

Great. Let's now read in our inital conditions and assign the appropraite units: 

In [2]:
x_c = np.loadtxt('../Data/Topography/REF_SurfTopo.dat')[:,0]    # x-coordinate
z_s = np.loadtxt('../Data/Topography/REF_SurfTopo.dat')[:,1]    # surface elevation (m a.s.l.)
z_b = np.loadtxt('../Data/Topography/REF_BedTopo.dat')[:,1]     # bed evelation (m a.s.l.)

H      = (z_s - z_b) * m                         # Ice thickness (m)
α      = np.zeros_like(H) * m/m                  # Surface slope (m/m)
# Surface slope only valid where H>0
α[H>0] = np.gradient(z_s[H>0], x_c[H>0])         

Now let's solve for the approximation of the basal shear stress ($\tau_b$) for "synthetic" little Kluane 

In [3]:
# Approximation of the basal shear stress for LK
𝜏 = ρ*g*H*np.sin(α)

Now let's solve for the $\beta$ corresponding to $u_b$ = 365 (m a$^{-1}$) or 1 (m d$^{-1}$). 

In [4]:
β_sqaured = 𝜏 / (365. * m /a)

In [5]:
print("β = {:.2e}".format(np.mean(β_sqaured[β_sqaured>0.0])))

β = 2.97e-04 megapascal * year / meter


In [6]:
np.sqrt(1 / 3.5e-4)

53.45224838248488